<a href="https://colab.research.google.com/github/jayantjain05/chat-bot/blob/main/main_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>